In [10]:
import sklearn

In [11]:
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier

import pandas as pd
import numpy as np

In [12]:
X, y = make_classification()
pdf = pd.DataFrame(X)
pdf.columns = ['c{}'.format(x) for x in range(X.shape[1])]

In [13]:
X.shape

(100, 20)

In [14]:
X1 = pdf[['c{}'.format(x) for x in range(10, 20)]]
X2 = pdf[['c{}'.format(x) for x in range(10)]]

In [19]:
class GraftingClassifier(SGDClassifier):
    """
    Currently only supports logistic regression.    
    """
    def __init__(self, loss="log", penalty='l2', alpha=0.0001, l1_ratio=0.15,
                 fit_intercept=True, max_iter=None, tol=None, shuffle=True,
                 verbose=0, epsilon=0.1, n_jobs=1,
                 random_state=None, learning_rate="optimal", eta0=0.0,
                 power_t=0.5, class_weight=None, warm_start=False,
                 average=False, n_iter=None, reg_penalty=None):
        super(GraftingClassifier, self).__init__(
            loss=loss, penalty=penalty, alpha=alpha, l1_ratio=l1_ratio,
            fit_intercept=fit_intercept, max_iter=max_iter, tol=tol,
            shuffle=shuffle, verbose=verbose, epsilon=epsilon, n_jobs=n_jobs,
            random_state=random_state, learning_rate=learning_rate, eta0=eta0,
            power_t=power_t, class_weight=class_weight, warm_start=warm_start,
            average=average, n_iter=n_iter)
        self.filter_cols = []
        self.base_shape = None
        self.reg_penalty = reg_penalty if reg_penalty is not None else l1_ratio
    
    def _fit_columns(self, X, return_x=True):
        """
        Method filter through "unselected" columns. The goal of this 
        method is to filter any uninformative columns.
        
        This will be selected based on index only?
        
        If return_x is false, it will only return the boolean mask.
        """
        import pandas
        bool_mask = np.ones((X.shape[1],), dtype=np.bool)
        if len(self.filter_cols) == 0:
            if return_x:
                return X
            else:
                return bool_mask
        # otherwise...
        bool_mask[self.filter_cols] = False
        if not return_x:
            return bool_mask
        if type(X) is pandas.core.frame.DataFrame:
            return X[X.columns[bool_mask]]
        else:
            return X[:, bool_mask]
    
    def _reg_penalty(self, tot_new_feats, base_size):
        remove_cols = np.argwhere(np.abs(self.coef_.flatten()[-tot_new_feats:]) < self.reg_penalty)
        add_cols = np.argwhere(np.abs(self.coef_.flatten()[-tot_new_feats:]) >= self.reg_penalty)
        base_coef = self.coef_.flatten()[:-tot_new_feats].tolist()
        # adding new coefs
        base_coef = base_coef + self.coef_.flatten()[-tot_new_feats:][add_cols].flatten().tolist()
        self.coef_ = np.array(base_coef).reshape(1, -1)
        remove_cols_offset = [base_size + x for x in remove_cols]
        self.filter_cols.append(remove_cols_offset)
        
        
    
    def _partial_grafting_fit(self, X_, y):
        """
        Partial fit grafting method to expand the coefficient listing
        to taking into account new coefficients
        """
        # require to know the base shape to determine/
        # check for irrelevant columns in the future.
        self.base_shape = self.coef_.shape[0]
        
        X = self._fit_columns(X_)
        n_samples, n_features = X.shape
        coef_list = np.zeros(n_features, dtype=np.float64, order="C")
        coef_list[:self.coef_.flatten().shape[0]] = self.coef_.flatten()
        self.coef_ = coef_list.reshape(1,-1)
        
    def partial_fit(self, X, y, sample_weight=None):
        base_size = len(self.filter_cols) + self.coef_.flatten().shape[0]
        tot_new_feats = X.shape[1] - base_size
        self._partial_grafting_fit(X, y)
        super(GraftingClassifier, self).partial_fit(X, y, sample_weight=None)  
        
        # update parameters based on weight of regularizer penalty
        self._reg_penalty(tot_new_feats, base_size)
        return self
    
    def predict(self, X):
        X = self._fit_columns(X)
        return super(GraftingClassifier, self).predict(X)
    
    def predict_proba(self, X):
        X = self._fit_columns(X)
        return super(GraftingClassifier, self).predict_proba(X)        

In [20]:
model = GraftingClassifier(max_iter=1000)
model.fit(X1, y)

GraftingClassifier(alpha=0.0001, average=False, class_weight=None,
          epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
          learning_rate='optimal', loss='log', max_iter=1000, n_iter=None,
          n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
          reg_penalty=0.15, shuffle=True, tol=None, verbose=0,
          warm_start=False)

In [21]:
model.coef_.shape

(1, 10)

In [22]:
model.partial_fit(pdf, y)

GraftingClassifier(alpha=0.0001, average=False, class_weight=None,
          epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
          learning_rate='optimal', loss='log', max_iter=1000, n_iter=None,
          n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
          reg_penalty=0.15, shuffle=True, tol=None, verbose=0,
          warm_start=False)

In [23]:
model.coef_.shape

(1, 9)

In [24]:
model.predict_proba(pdf)

array([[ 0.00619828,  0.99380172],
       [ 0.92969369,  0.07030631],
       [ 0.68502224,  0.31497776],
       [ 0.11524396,  0.88475604],
       [ 0.03516324,  0.96483676],
       [ 0.35131229,  0.64868771],
       [ 0.72360682,  0.27639318],
       [ 0.7947508 ,  0.2052492 ],
       [ 0.18609653,  0.81390347],
       [ 0.17042461,  0.82957539],
       [ 0.94364955,  0.05635045],
       [ 0.72174869,  0.27825131],
       [ 0.98252103,  0.01747897],
       [ 0.60455504,  0.39544496],
       [ 0.77980623,  0.22019377],
       [ 0.69483036,  0.30516964],
       [ 0.5413798 ,  0.4586202 ],
       [ 0.3501247 ,  0.6498753 ],
       [ 0.36370671,  0.63629329],
       [ 0.5962762 ,  0.4037238 ],
       [ 0.419771  ,  0.580229  ],
       [ 0.89638916,  0.10361084],
       [ 0.75839285,  0.24160715],
       [ 0.82523315,  0.17476685],
       [ 0.44215415,  0.55784585],
       [ 0.05100563,  0.94899437],
       [ 0.20269502,  0.79730498],
       [ 0.62765282,  0.37234718],
       [ 0.72584456,